In [91]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import csv
from datetime import datetime

* Mood vector Design 1: for each day, if negative_count > positive_count or negative_count > neutral_count or neative_count > mix_count, assign negative to that day; if positive_count > negative_count or positive_count > neutral_count or positive_count > mix_count, assign positive to that day; if mix_count >= positive_count or mix_count >= negative_count or negative_count == positive_count, assign mix to that day; if neutral_count >= positive_count or neutral_count >= negative_count or neutral_count > mix_count, assign neutral to that day.

In [153]:
#path = '/Users/lucia/phd_work/cognitive_distortion/'
path = '/home/lucia/phd_work/mypersonality_data/cognitive_distortion/'
AllP = pd.read_csv(path + '/data/important_data/twoM_newLabels2.csv')
ids = pd.read_csv(path + '/data/important_data/FinalSampleUsers.csv')
#remove underage and ppl with less than 80% of posts retained
AllP  = AllP [AllP ['userid'].isin(ids['userid'])]
print(AllP.shape)
lyrics = pd.read_csv(path + '/data/important_data/lyrics/QuotesDetected_all.csv')
lyrics = lyrics[['text','label']]
time = pd.merge(lyrics, AllP, on='text', how = 'right')
print(time.shape)
time['label'] = time['label'].replace(np.nan, 'NotQuote', regex=True)
time = time[['text', 'userid', 'time', 'time_diff', 'id', 'label']]
time['label'].replace(['NotQuote','quote','suspect'],[2,1,1],inplace=True)


(3838, 14)
(3838, 15)


In [155]:
len(time[time['label'] == 2])

3577

In [156]:
#select date
time['time'] = time['time'].apply(lambda x: x.split()[0])
time[0:5]

,text,userid,time,time_diff,id,label
0,"I've got an ace up my sleeve, and a jack in my...",8ff6192bbbb717c36097b33e244d91de,04/02/2011,53,3055,2
1,I've got cold feet.,f49956142aab2ad74decfd49a103c2f1,12/01/2011,38,904,2
2,"I've got hoes I've got hoes, in different area...",009d96a823b6f6c085c092fb177491f6,16/03/2011,57,957,2
3,I've got the electric tea kettle. I need tea a...,f49956142aab2ad74decfd49a103c2f1,23/12/2010,58,906,2
4,I've gotten up from my desk three times to get...,0e3fdb89c7706b13891170ab2a9017c7,18/03/2011,5,1128,2


In [157]:
#convert to time series
# time['time'] = time['time'].apply(lambda x: parser.parse(x))
# time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d %b %Y'))
time['time'] = time['time'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
time['time'][1:10]

1   2011-01-12
2   2011-03-16
3   2010-12-23
4   2011-03-18
5   2011-01-07
6   2011-03-10
7   2011-05-11
8   2011-02-07
9   2011-01-11
Name: time, dtype: datetime64[ns]

In [158]:
#sort values according to userid, time and valence
time = time.sort_values(by=['userid','time_diff','label'],  ascending=False)
#time.to_csv(path + '/data/important_data/time.csv')

In [173]:
def getTransitions(LyricObject):
    emptyTran = 0
    lyricsTran = 0
    lyricsEmpty = 0
    Original = 0
    LyricsOriginal = 0 
    OriginalEmpty = 0 
    preValence = 0
    for valence in LyricObject:
    #these are self transition states
        if valence == 0 and preValence == 0:
            emptyTran = emptyTran + 1
            
        elif valence == 1 and preValence == 1:
            lyricsTran = lyricsTran + 1
            
        elif (valence == 0 and preValence) == 1 or (valence == 1 and preValence == 0):
            lyricsEmpty = lyricsEmpty + 1
            
        elif valence == 2 and preValence == 2:
            Original = Original + 1
            
        elif (valence == 1 and preValence == 2) or  (valence == 2 and preValence == 1):
            LyricsOriginal = LyricsOriginal + 1
            
        elif (valence == 2 and preValence == 0) or  (valence == 0 and preValence == 2):
            OriginalEmpty = OriginalEmpty + 1 
            
        preValence = valence
  
    return [emptyTran, lyricsTran, lyricsEmpty, Original, LyricsOriginal, OriginalEmpty, preValence]


all the users should start with 60 days, if not, that's because their posts contain foreign lanaguge and they were cleaned from the data set, in this case, the valence vector starts with the first post


In [160]:
def getLyricsVector(df):
    valenceVec = {}
    valences =[]
    preUser = None
    preDay = None
    preValence = []

    for valence, day, user in zip(df['label'], df['time_diff'], df['userid']):
        if preUser is None:
            if day < 60: #if first day is not 60 
                addDays = 60 - day
                for num in range(0, addDays):
                    valences.append(0)
                valenceVec[user] = valences
            else:
                valences = [valence]
                valenceVec[user] = valences

        elif user == preUser and preDay == day+1:
            valences.append(valence)
            
        elif user == preUser and preDay != day+1:
            valences.append(valence)
            addDays = preDay - (day+1)
            for num in range(0, addDays):
                valences.append(0)
                
        elif user != preUser:
            if day < 60: #if first day is not 60 
                addDays = 60 - day
               # print(user,addDays)
                valences = [valence]
                for num in range(0, addDays):
                    valences.insert(0, 0)
                valenceVec[user] = valences
                #print(valences)
            else:
                valences = [valence]
                valenceVec[user] = valences
                #print(valences)
                
            if preDay > 1:
                addDays = preDay
                for num in range(0, addDays):
                    preValence.append(0)
                    
        preUser = user
        preDay = day
        preValence = valences
#     valenceVec[preUser] = valences
    return valenceVec

valenceVec = getLyricsVector(time)

In [161]:
i = 0
for item in valenceVec:
    print(item, valenceVec[item])
    i = i+1
print(i)


f8708568eff1535c85247b28dc126f9b [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
f49956142aab2ad74decfd49a103c2f1 [0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0]
ec3dc451b8a811f05ace158a0d76e32e [0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 1, 2, 0, 0, 2, 2, 2

In [184]:
def countLyrics(vector,val):
    count = 0
    mylist = []
    for userid in valenceVec:
        mylist.append(valenceVec[userid].count(val))
    return mylist


empty = countLyrics(valenceVec,0)
lyrics = countLyrics(valenceVec,1)
original = countLyrics(valenceVec,2)


In [174]:
def getUserTransitions(valencVec):
    result = {}
    for item in valencVec:
        result[item] = getTransitions(valencVec[item])
#         print(result)
    return result
        
TransitionStates = getUserTransitions(valenceVec)   


In [194]:
len(original)

70

create table for statistics

In [185]:
df = pd.DataFrame(np.array(empty).reshape(70,1), columns=['emptyDays'])
df['lyrics'] = lyrics
df['original'] = original
df['userid'] = valenceVec.keys()

In [186]:
df.describe()

,emptyDays,lyrics,original
count,70.000000,70.000000,70.000000
mean,29.071429,3.728571,51.100000
std,11.636177,4.866095,37.719464
min,0.000000,0.000000,17.000000
25%,23.250000,1.000000,26.500000
50%,30.500000,2.000000,39.500000
75%,37.750000,4.750000,53.000000
max,44.000000,36.000000,204.000000


In [63]:
# #pickle
# import pickle
# with open(path + 'newScripts/moodVector/moodVectorsData/MoodVecDes3Empty.pickle', 'wb') as handle:
#     pickle.dump(valenceVec, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [187]:
#write result as csv
#header = ['userid','vector']
with open(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsTransitions(empty).csv','w') as csv_file:
    writer = csv.writer(csv_file)
   # writer.writerow(i for i in header)
    writer.writerow(TransitionStates.keys())
    for row in zip(*TransitionStates.values()):
        writer.writerow(list(row))

In [188]:
#read csv file 
file = pd.read_csv(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsTransitions(empty).csv')
file = file.transpose()
file.columns = ['emptyTran', 'lyricsTran', 'lyricsEmpty', 'Original', 'LyricsOriginal', 'OriginalEmpty', 'preValence']
file.shape


(70, 7)

In [189]:
#add one column as row sum
file['allTrans'] = file.sum(axis=1) 
file[1:10]

,emptyTran,lyricsTran,lyricsEmpty,Original,LyricsOriginal,OriginalEmpty,preValence,allTrans
f49956142aab2ad74decfd49a103c2f1,19,0,0,38,0,22,0,79
ec3dc451b8a811f05ace158a0d76e32e,36,0,0,10,6,15,2,69
eaeabcfe25866f0455d349bae2450d88,12,0,0,30,0,31,2,75
e9970fe8848e9505b52d414d81611de8,28,0,9,13,4,16,1,71
e076953cd3fcf8927b8f21574c0890f3,14,0,3,23,3,25,0,68
dcbe08425f5edef5e5c1053f21e83640,30,3,6,8,6,16,0,69
db7f1b0130b2138e13b7bbca9cc63823,14,1,3,24,9,26,2,79
d5380466e18422d7b2c2d54d57cbbde2,23,0,0,12,6,28,0,69
cfb021cfbff46ffdb75c7dcafe7c2087,13,1,1,34,16,20,1,86


compute transition states probability,  we compute the pobability by dividing the transition with number of all posts

In [190]:
Tranprob = file.apply(lambda x: x/file.iloc[:,-1])
#Tranprob = file.apply(lambda x: x/1)
Tranprob['userid'] = Tranprob.index
Tranprob[1:20]

,emptyTran,lyricsTran,lyricsEmpty,Original,LyricsOriginal,OriginalEmpty,preValence,allTrans,userid
f49956142aab2ad74decfd49a103c2f1,0.240506,0.000000,0.000000,0.481013,0.000000,0.278481,0.000000,1.0,f49956142aab2ad74decfd49a103c2f1
ec3dc451b8a811f05ace158a0d76e32e,0.521739,0.000000,0.000000,0.144928,0.086957,0.217391,0.028986,1.0,ec3dc451b8a811f05ace158a0d76e32e
eaeabcfe25866f0455d349bae2450d88,0.160000,0.000000,0.000000,0.400000,0.000000,0.413333,0.026667,1.0,eaeabcfe25866f0455d349bae2450d88
e9970fe8848e9505b52d414d81611de8,0.394366,0.000000,0.126761,0.183099,0.056338,0.225352,0.014085,1.0,e9970fe8848e9505b52d414d81611de8
e076953cd3fcf8927b8f21574c0890f3,0.205882,0.000000,0.044118,0.338235,0.044118,0.367647,0.000000,1.0,e076953cd3fcf8927b8f21574c0890f3
dcbe08425f5edef5e5c1053f21e83640,0.434783,0.043478,0.086957,0.115942,0.086957,0.231884,0.000000,1.0,dcbe08425f5edef5e5c1053f21e83640
db7f1b0130b2138e13b7bbca9cc63823,0.177215,0.012658,0.037975,0.303797,0.113924,0.329114,0.025316,1.0,db7f1b0130b2138e13b7bbca9cc63823
d5380466e18422d7b2c2d54d57cbbde2,0.333333,0.000000,0.000000,0.173913,0.086957,0.405797,0.000000,1.0,d5380466e18422d7b2c2d54d57cbbde2
cfb021cfbff46ffdb75c7dcafe7c2087,0.151163,0.011628,0.011628,0.395349,0.186047,0.232558,0.011628,1.0,cfb021cfbff46ffdb75c7dcafe7c2087
cea00424ced413c108d878b1a14f3316,0.253968,0.000000,0.015873,0.285714,0.015873,0.428571,0.000000,1.0,cea00424ced413c108d878b1a14f3316


In [68]:
#Tranprob.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsValenceTransitions(Empty).csv')

correlation with all scales

In [191]:
allData = pd.read_csv( path + '/data/important_data/user_scale_post_time2.csv')
Var = allData[['userid','ope','con','ext','agr','neu','swl','CESD_sum']]
Var.shape

(301, 8)

In [192]:
compare = pd.merge(Tranprob, Var, on ='userid', how = 'inner')
#compare
compare.to_csv(path +'newScripts/moodVector/moodVectorsData/PostsLyricsTranEmptyAllVar.csv')
corMatrix = compare.corr()
corMatrix.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsTrancorrelationMatrix(Empty).csv')
corMatrix

,emptyTran,lyricsTran,lyricsEmpty,Original,LyricsOriginal,OriginalEmpty,preValence,allTrans,ope,con,ext,agr,neu,swl,CESD_sum
emptyTran,1.000000,-0.055758,0.287184,-0.843094,-0.327379,0.354876,-0.211947,NaN,0.091494,-0.054387,0.056070,0.150688,-0.075991,0.043099,-0.056801
lyricsTran,-0.055758,1.000000,0.310077,-0.094607,0.504198,-0.181418,0.063725,NaN,-0.247327,-0.097041,-0.094732,-0.069997,0.248625,-0.093845,0.263566
lyricsEmpty,0.287184,0.310077,1.000000,-0.426229,0.132119,0.092729,-0.068877,NaN,0.087421,-0.089254,0.068708,-0.017715,-0.009918,-0.069460,0.083008
Original,-0.843094,-0.094607,-0.426229,1.000000,0.094363,-0.699489,0.077224,NaN,-0.113349,0.021139,-0.082772,-0.144441,0.139353,-0.052317,0.039473
LyricsOriginal,-0.327379,0.504198,0.132119,0.094363,1.000000,-0.303320,0.067861,NaN,-0.129093,-0.087792,-0.029832,-0.215124,0.180273,-0.201605,0.266387
OriginalEmpty,0.354876,-0.181418,0.092729,-0.699489,-0.303320,1.000000,0.017724,NaN,0.165134,0.102531,0.084057,0.187928,-0.276956,0.157193,-0.194533
preValence,-0.211947,0.063725,-0.068877,0.077224,0.067861,0.017724,1.000000,NaN,-0.026893,0.067473,0.068923,-0.031025,-0.001034,0.075720,0.179741
allTrans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ope,0.091494,-0.247327,0.087421,-0.113349,-0.129093,0.165134,-0.026893,NaN,1.000000,-0.085620,0.294009,0.037679,-0.193090,-0.069822,-0.196169
con,-0.054387,-0.097041,-0.089254,0.021139,-0.087792,0.102531,0.067473,NaN,-0.085620,1.000000,0.187503,0.147915,-0.361895,0.409762,-0.393027


In [193]:
compareFreq = pd.merge(df, Var, on ='userid', how = 'left')
corMatrix = compareFreq.corr()
corMatrix.to_csv(path + 'newScripts/moodVector/moodVectorsData/PostsLyricsFreqMatrix(Empty).csv')
corMatrix

,emptyDays,lyrics,original,ope,con,ext,agr,neu,swl,CESD_sum
emptyDays,1.000000,-0.333415,-0.922294,0.158735,-0.014964,0.061271,0.136232,-0.160577,0.055726,-0.081643
lyrics,-0.333415,1.000000,0.257558,-0.177018,-0.122889,-0.060171,-0.127256,0.252452,-0.229339,0.292355
original,-0.922294,0.257558,1.000000,-0.151762,-0.035444,-0.073394,-0.134167,0.147683,-0.081345,0.090313
ope,0.158735,-0.177018,-0.151762,1.000000,-0.085620,0.294009,0.037679,-0.193090,-0.069822,-0.196169
con,-0.014964,-0.122889,-0.035444,-0.085620,1.000000,0.187503,0.147915,-0.361895,0.409762,-0.393027
ext,0.061271,-0.060171,-0.073394,0.294009,0.187503,1.000000,0.224412,-0.337890,0.209976,-0.342366
agr,0.136232,-0.127256,-0.134167,0.037679,0.147915,0.224412,1.000000,-0.287761,0.233524,-0.208865
neu,-0.160577,0.252452,0.147683,-0.193090,-0.361895,-0.337890,-0.287761,1.000000,-0.406282,0.604651
swl,0.055726,-0.229339,-0.081345,-0.069822,0.409762,0.209976,0.233524,-0.406282,1.000000,-0.481023
CESD_sum,-0.081643,0.292355,0.090313,-0.196169,-0.393027,-0.342366,-0.208865,0.604651,-0.481023,1.000000


Now we write the valence vector as csv file 